## Imports

In [2]:
import xpress as xp
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns

In [3]:
%load_ext line_profiler

In [4]:
%matplotlib widget

In [5]:
xp.controls.outputlog = 1
xp.controls.threads = 1
xp.controls.scaling = 0
xp.controls.presolve = 0
xp.controls.feastol = 1.e-7
xp.controls.optimalitytol = 1.e-7
xp.setOutputEnabled(True)

In [6]:
def retrieve_problem(year,week,output_path,itr=1):
    model = xp.problem()
    model.read(output_path+f"/problem-{year}-{week}--optim-nb-{itr}.mps")
    return(model)

## Comparaison initial/separate

In [6]:
# study_path = "D:/Users/gerbauxjul/Documents/6-Etudes Antares/bug_milp"
study_path = "Z:/JulietteG/6-Etutes_Antares/H2_france/BP23_A_saME_Xeq_2031_mps/output"

In [7]:
output_path_initial = study_path+"/20241128-1728eco-initial_simulation"
output_path_separate = study_path+"/20241128-1732eco-separate_fr_eu"

In [ ]:
m_initial = retrieve_problem(77,1,output_path_initial,itr=1)
m_separate = retrieve_problem(77,1,output_path_separate,itr=1)

Reading Problem Pb Solve
Problem Statistics
       95295 (      1 spare) rows
      227304 (      0 spare) structural columns
      500472 ( 171360 spare) non-zero elements
Global Statistics
           0 entities        0 sets        0 set members


In [ ]:
m_initial.write("problem_initial.lp","l")
m_separate.write("problem_separate.lp","l")

## Upper bound avec Xpansion

In [7]:
# study_path = "D:/Users/gerbauxjul/Documents/6-Etudes Antares/bug_milp"
study_path = "Z:/JulietteG/6-Etutes_Antares/H2_france/BP23_A_ref/output"

In [1]:
output_path_initial = study_path+"/20250108-1706eco-mps_xpansion"

NameError: name 'study_path' is not defined

In [8]:
m_initial = retrieve_problem(67,1,output_path_initial,itr=1)

: 

In [9]:
m_initial.write("problem_upper_bound.lp","l")

In [10]:
m_initial.solve()

FICO Xpress v8.14.8, Hyper, solve started 11:20:40, Jan 9, 2025
Heap usage: 83MB (peak 83MB, 19MB system)
Minimizing LP Pb Solve using 1 thread and up to 31GB memory, with these control settings:
SCALING = 0
PRESOLVE = 0
OUTPUTLOG = 1
THREADS = 1
FEASTOL = 1.e-7
OPTIMALITYTOL = 1.e-7
Original problem has:
     95976 rows       228749 cols       505445 elements

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 8.00e-02,  1.92e+00] / [ 8.00e-02,  1.92e+00]
  RHS and bounds [min,max] : [ 3.00e-09,  2.65e+06] / [ 3.00e-09,  2.65e+06]
  Objective      [min,max] : [ 5.00e-04,  1.61e+04] / [ 5.00e-04,  1.61e+04]

Crash basis containing 4428 structural columns created
 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0      -1503047668.      D      1  5208       2.880797     0
 38461      -1187910132.      D  24390     0        .000000     2
 52286      -1069086773.      D  17812     0        .000000     4
 60272

In [11]:
m_initial.getObjVal() #-1.0182913501#-1.0182913501e+09

-1018291350.0988952

In [12]:
vars = m_initial.getVariable()
names_ref = [v.name for v in vars]

In [15]:
def create_dataframe_variables(m,names):
    idx = [i for i in range(len(names))]
    vars = m.getVariable()
    df_vars = pd.DataFrame([vars[i] for i in idx], columns=["var"])
    df_vars["names"] = [names[i] for i in idx]
    df_vars["split"] = df_vars["names"].apply(lambda x:x.strip().split("::"))
    df_vars["name_var"] = df_vars["split"].apply(lambda x:x[0])
    df_vars["antares_object"] = df_vars["split"].apply(lambda x:x[1].split("<")[0])
    df_vars["name_antares_object"] = df_vars["split"].apply(lambda x:x[1].split("<")[1].split(">")[0])
    df_vars["subobject"] = df_vars["split"].apply(lambda x:x[2] if len(x)>=4 else "None")
    df_vars["time"] = df_vars["split"].apply(lambda x:int(x[-1].split("<")[1].split(">")[0]))
# coltype = []
# m.getcoltype(coltype, 0, m.attributes.cols - 1)
# df_vars['coltype'] = coltype
    # bounds = []
    # m.getlb(bounds, 0, m.attributes.cols - 1)
    # df_vars['lb'] = [bounds[i] for i in idx]
    # bounds = []
    # m.getub(bounds, 0, m.attributes.cols - 1)
    # df_vars['ub'] = [bounds[i] for i in idx]
    # df_vars = df_vars.assign(sol=[m.getSolution(i) for i in idx])
    # objcoef = []
    # m.getobj(objcoef,0,m.attributes.cols-1)
    # df_vars= df_vars.assign(coeff=[objcoef[i] for i in range(m.attributes.cols)])
    # df_vars = df_vars.assign(sol=[m.getSolution(i) for i in range(m.attributes.cols)])
    return df_vars

In [14]:
df_vars_itr1 = create_dataframe_variables(m_initial,names_ref)

In [15]:
df_vars_itr1["name_var"].unique()

array(['NTCDirect', 'IntercoDirectCost', 'IntercoIndirectCost',
       'DispatchableProduction', 'PositiveUnsuppliedEnergy',
       'NegativeUnsuppliedEnergy', 'HydProd', 'Pumping', 'HydroLevel',
       'Overflow', 'FinalStorage', 'LayerStorage'], dtype=object)

In [16]:
df_vars_itr1[df_vars_itr1["name_var"]=='LayerStorage']

,var,names,split,name_var,antares_object,name_antares_object,subobject,time
228649,LayerStorage::area<v_me_h2_long>::Layer<0>::ho...,LayerStorage::area<v_me_h2_long>::Layer<0>::ho...,"[LayerStorage, area<v_me_h2_long>, Layer<0>, h...",LayerStorage,area,v_me_h2_long,Layer<0>,167
228650,LayerStorage::area<v_me_h2_long>::Layer<1>::ho...,LayerStorage::area<v_me_h2_long>::Layer<1>::ho...,"[LayerStorage, area<v_me_h2_long>, Layer<1>, h...",LayerStorage,area,v_me_h2_long,Layer<1>,167
228651,LayerStorage::area<v_me_h2_long>::Layer<2>::ho...,LayerStorage::area<v_me_h2_long>::Layer<2>::ho...,"[LayerStorage, area<v_me_h2_long>, Layer<2>, h...",LayerStorage,area,v_me_h2_long,Layer<2>,167
228652,LayerStorage::area<v_me_h2_long>::Layer<3>::ho...,LayerStorage::area<v_me_h2_long>::Layer<3>::ho...,"[LayerStorage, area<v_me_h2_long>, Layer<3>, h...",LayerStorage,area,v_me_h2_long,Layer<3>,167
228653,LayerStorage::area<v_me_h2_long>::Layer<4>::ho...,LayerStorage::area<v_me_h2_long>::Layer<4>::ho...,"[LayerStorage, area<v_me_h2_long>, Layer<4>, h...",LayerStorage,area,v_me_h2_long,Layer<4>,167
...,...,...,...,...,...,...,...,...
228744,LayerStorage::area<v_me_h2_long>::Layer<95>::h...,LayerStorage::area<v_me_h2_long>::Layer<95>::h...,"[LayerStorage, area<v_me_h2_long>, Layer<95>, ...",LayerStorage,area,v_me_h2_long,Layer<95>,167
228745,LayerStorage::area<v_me_h2_long>::Layer<96>::h...,LayerStorage::area<v_me_h2_long>::Layer<96>::h...,"[LayerStorage, area<v_me_h2_long>, Layer<96>, ...",LayerStorage,area,v_me_h2_long,Layer<96>,167
228746,LayerStorage::area<v_me_h2_long>::Layer<97>::h...,LayerStorage::area<v_me_h2_long>::Layer<97>::h...,"[LayerStorage, area<v_me_h2_long>, Layer<97>, ...",LayerStorage,area,v_me_h2_long,Layer<97>,167
228747,LayerStorage::area<v_me_h2_long>::Layer<98>::h...,LayerStorage::area<v_me_h2_long>::Layer<98>::h...,"[LayerStorage, area<v_me_h2_long>, Layer<98>, ...",LayerStorage,area,v_me_h2_long,Layer<98>,167


In [17]:
sol_layer = [m_initial.getSolution(i) for i in df_vars_itr1[df_vars_itr1["name_var"]=='LayerStorage'].index]

In [18]:
objcoef = []
m_initial.getobj(objcoef,228649,228748)
print(len(objcoef))

100


In [19]:
cost_bellman = sum([sol_layer[i]*objcoef[i] for i in range(100)])
print(cost_bellman)

-1831550909.6311038


In [20]:
m_initial.getObjVal() - cost_bellman

813259559.5322086

In [8]:
output_path_initial = study_path+"/20250109-1742eco-mps_xpansion"

In [9]:
m_initial = retrieve_problem(67,1,output_path_initial,itr=1)

Reading Problem Pb Solve
Problem Statistics
       95976 (      1 spare) rows
      228749 (      0 spare) structural columns
      505445 ( 172468 spare) non-zero elements
Global Statistics
           0 entities        0 sets        0 set members


In [10]:
m_initial.write("problem_upper_bound_bis.lp","l")

In [11]:
m_initial.solve()

FICO Xpress v8.14.8, Hyper, solve started 15:24:55, Jan 10, 2025
Heap usage: 83MB (peak 83MB, 19MB system)
Minimizing LP Pb Solve using 1 thread and up to 31GB memory, with these control settings:
SCALING = 0
PRESOLVE = 0
OUTPUTLOG = 1
THREADS = 1
FEASTOL = 1.e-7
OPTIMALITYTOL = 1.e-7
Original problem has:
     95976 rows       228749 cols       505445 elements

Coefficient range                    original                 solved        
  Coefficients   [min,max] : [ 8.00e-02,  1.92e+00] / [ 8.00e-02,  1.92e+00]
  RHS and bounds [min,max] : [ 3.00e-09,  2.65e+06] / [ 3.00e-09,  2.65e+06]
  Objective      [min,max] : [ 5.00e-04,  1.61e+04] / [ 5.00e-04,  1.61e+04]

Crash basis containing 4428 structural columns created
 
   Its         Obj Value      S   Ninf  Nneg   Sum Dual Inf  Time
     0      -1495728318.      D      1  5208       2.880797     0
 38461      -1180609259.      D  24390     0        .000000     3
 52286      -1061793958.      D  17812     0        .000000     6
 6026

In [12]:
m_initial.getObjVal() #-1.0109858763e+09

-1010985876.2534821

In [13]:
vars = m_initial.getVariable()
names_ref = [v.name for v in vars]

In [16]:
df_vars_itr1 = create_dataframe_variables(m_initial,names_ref)

In [17]:
df_vars_itr1["name_var"].unique()

array(['NTCDirect', 'IntercoDirectCost', 'IntercoIndirectCost',
       'DispatchableProduction', 'PositiveUnsuppliedEnergy',
       'NegativeUnsuppliedEnergy', 'HydProd', 'Pumping', 'HydroLevel',
       'Overflow', 'FinalStorage', 'LayerStorage'], dtype=object)

In [18]:
df_vars_itr1[df_vars_itr1["name_var"]=='LayerStorage']

,var,names,split,name_var,antares_object,name_antares_object,subobject,time
228649,LayerStorage::area<v_me_h2_long>::Layer<0>::ho...,LayerStorage::area<v_me_h2_long>::Layer<0>::ho...,"[LayerStorage, area<v_me_h2_long>, Layer<0>, h...",LayerStorage,area,v_me_h2_long,Layer<0>,167
228650,LayerStorage::area<v_me_h2_long>::Layer<1>::ho...,LayerStorage::area<v_me_h2_long>::Layer<1>::ho...,"[LayerStorage, area<v_me_h2_long>, Layer<1>, h...",LayerStorage,area,v_me_h2_long,Layer<1>,167
228651,LayerStorage::area<v_me_h2_long>::Layer<2>::ho...,LayerStorage::area<v_me_h2_long>::Layer<2>::ho...,"[LayerStorage, area<v_me_h2_long>, Layer<2>, h...",LayerStorage,area,v_me_h2_long,Layer<2>,167
228652,LayerStorage::area<v_me_h2_long>::Layer<3>::ho...,LayerStorage::area<v_me_h2_long>::Layer<3>::ho...,"[LayerStorage, area<v_me_h2_long>, Layer<3>, h...",LayerStorage,area,v_me_h2_long,Layer<3>,167
228653,LayerStorage::area<v_me_h2_long>::Layer<4>::ho...,LayerStorage::area<v_me_h2_long>::Layer<4>::ho...,"[LayerStorage, area<v_me_h2_long>, Layer<4>, h...",LayerStorage,area,v_me_h2_long,Layer<4>,167
...,...,...,...,...,...,...,...,...
228744,LayerStorage::area<v_me_h2_long>::Layer<95>::h...,LayerStorage::area<v_me_h2_long>::Layer<95>::h...,"[LayerStorage, area<v_me_h2_long>, Layer<95>, ...",LayerStorage,area,v_me_h2_long,Layer<95>,167
228745,LayerStorage::area<v_me_h2_long>::Layer<96>::h...,LayerStorage::area<v_me_h2_long>::Layer<96>::h...,"[LayerStorage, area<v_me_h2_long>, Layer<96>, ...",LayerStorage,area,v_me_h2_long,Layer<96>,167
228746,LayerStorage::area<v_me_h2_long>::Layer<97>::h...,LayerStorage::area<v_me_h2_long>::Layer<97>::h...,"[LayerStorage, area<v_me_h2_long>, Layer<97>, ...",LayerStorage,area,v_me_h2_long,Layer<97>,167
228747,LayerStorage::area<v_me_h2_long>::Layer<98>::h...,LayerStorage::area<v_me_h2_long>::Layer<98>::h...,"[LayerStorage, area<v_me_h2_long>, Layer<98>, ...",LayerStorage,area,v_me_h2_long,Layer<98>,167


In [19]:
sol_layer = [m_initial.getSolution(i) for i in df_vars_itr1[df_vars_itr1["name_var"]=='LayerStorage'].index]

In [20]:
objcoef = []
m_initial.getobj(objcoef,228649,228748)
print(len(objcoef))

100


In [21]:
final_level = sum([sol_layer[i] for i in range(100)])
print(final_level/363000*100)

58.253847028412274


In [22]:
cost_bellman = sum([sol_layer[i]*objcoef[i] for i in range(100)])
print(cost_bellman) #1822006143
print((cost_bellman+1822006143)/-1822006143*100)

-1824245435.7856967
0.12290259252417596


In [23]:
m_initial.getObjVal() - cost_bellman

813259559.5322146

In [1]:
(m_initial.getObjVal() - cost_bellman-813258218)/813258218*100 #813258218

NameError: name 'm_initial' is not defined